<a href="https://colab.research.google.com/github/monishramadoss/ofa_quant/blob/main/ofa_quant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ofa

     |████████████████████████████████| 106 kB 2.1 MB/s 
